<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/DP_FL_Not_Division_Dataset_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.9 

In [2]:
#Importanto as bibliotecas necessárias
from collections import OrderedDict
from typing import Dict, Optional, List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
import opacus
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [3]:
#Definindo o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__}, Flower{fl.__version__} and Opacus{opacus.__version__ }")

Training on cpu using Pytorch2.0.1+cu118, Flower1.4.0 and Opacus1.4.0


In [4]:
# Definindo os valores das variáveis
BATCH_SIZE = 1024
NUM_CLIENTS= 10
optim_lr=0.002
local_epochs=5
num_rounds=10
noise_multiplier=1.1
max_grad_norm=1.2


In [5]:
# Carregando os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
    testloader = DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)
    num_examples = {"trainset": len(trainset)*BATCH_SIZE, "testset": len(testset)*BATCH_SIZE}

    return trainloader, testloader, num_examples

In [6]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
# Definindo as funções de treino e teste
def train(net, trainloader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    privacy_engine = PrivacyEngine()
    net, optimizer, trainloader = privacy_engine.make_private(
       module=net,
       optimizer=optimizer,
       data_loader=trainloader,
       noise_multiplier=noise_multiplier,
       max_grad_norm=max_grad_norm, )
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [8]:
# Funções de atualização de parâmetros do modelo
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [9]:

# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, testloader,num_examples) -> None:
        super().__init__()
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader
        self.num_examples = num_examples


    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}


    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader, testloader, num_examples = load_data()
  
        return FlowerClient(cid, net, trainloader, testloader,num_examples)

In [10]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [11]:

# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-10 06:31:55,986 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-10 06:31:58,944	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 06:32:03,336 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7775195136.0, 'object_store_memory': 3887597568.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7775195136.0, 'object_store_memory': 3887597568.0}
INFO flwr 2023-06-10 06:32:03,349 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 06:32:03,369 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=887) 2023-06-10 06:32:10.373953: W tensorflow/compiler/

(launch_and_get_parameters pid=887) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 75960906.94it/s]


(launch_and_get_parameters pid=887) Extracting ./data/cifar-10-python.tar.gz to ./data
(launch_and_get_parameters pid=887) Files already downloaded and verified


INFO flwr 2023-06-10 06:32:23,041 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-06-10 06:32:23,045 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-06-10 06:32:23,051 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-10 06:32:23,053 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 1: strategy sampled 10 clients (out of 10)


...

(launch_and_fit pid=887) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=887) Files already downloaded and verified
(launch_and_fit pid=886) [Client 3] fit, config: {}
(launch_and_fit pid=886) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=887) Files already downloaded and verified
(launch_and_fit pid=886) [Client 8] fit, config: {}
(launch_and_fit pid=887) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_fit pid=887) [Client 4] fit, config: {}
(launch_and_fit pid=886) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_fit pid=886) [Client 1] fit, config: {}
(launch_and_fit pid=887) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_fit pid=887) [Client 6] fit, config: {}
(launch_and_fit pid=886) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_fit pid=886) [Client 0] fit, config: {}
(laun

DEBUG flwr 2023-06-10 14:44:37,325 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-06-10 14:44:37,374 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=887) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_evaluate pid=887) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=887) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=886) Files already downloaded and verified
(launch_and_evaluate pid=887) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=887) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=886) Files already downloaded and verified
(launch_and_evaluate pid=887) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=887) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=886) Files already downloaded and verified
(launch_and_evaluate pid=886) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=886) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=887) Files already downloaded and veri

DEBUG flwr 2023-06-10 14:45:41,253 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-06-10 14:45:41,257 | server.py:147 | FL finished in 29598.203859329
INFO:flwr:FL finished in 29598.203859329
INFO flwr 2023-06-10 14:45:41,263 | app.py:218 | app_fit: losses_distributed [(1, 0.0018218660831451413), (2, 0.0017821322083473204), (3, 0.001775456583499908), (4, 0.001785389792919159), (5, 0.0017881004214286807), (6, 0.0017936610341072082), (7, 0.0017819411039352417), (8, 0.0017879519343376163), (9, 0.0017890381336212154), (10, 0.0018002925038337705)]
INFO:flwr:app_fit: losses_distributed [(1, 0.0018218660831451413), (2, 0.0017821322083473204), (3, 0.001775456583499908), (4, 0.001785389792919159), (5, 0.0017881004214286807), (6, 0.0017936610341072082), (7, 0.0017819411039352417), (8, 0.0017879519343376163), (9, 0.0017890381336212154), (10, 0.0018002925038337705)]
INFO flwr 2023-06-10 14:45:41,2

History (loss, distributed):
	round 1: 0.0018218660831451413
	round 2: 0.0017821322083473204
	round 3: 0.001775456583499908
	round 4: 0.001785389792919159
	round 5: 0.0017881004214286807
	round 6: 0.0017936610341072082
	round 7: 0.0017819411039352417
	round 8: 0.0017879519343376163
	round 9: 0.0017890381336212154
	round 10: 0.0018002925038337705
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.3625), (2, 0.4096000000000001), (3, 0.4355000000000001), (4, 0.45099999999999985), (5, 0.4676000000000001), (6, 0.47740000000000005), (7, 0.48909999999999987), (8, 0.49660000000000004), (9, 0.5047), (10, 0.5127000000000002)]}